Author: Kevin ALBERT  

Created: October 2021

In [1]:
import datetime, time
print ('Last testrun on: ' + datetime.datetime.now().strftime("%d %b %Y"))

Last testrun on: 21 Oct 2021


# Cardano
_**how to use tools for interacting with the Cardano blockchain platform using Python**_  

## Contents  
  * [example with REST protocol](#REST-protocol-example)
  * [example with python module](#Python-module-example)

### setup

```bash
mkdir ~/notebooks/cardano
cd ~/notebooks/cardano
wget https://github.com/input-output-hk/cardano-wallet/raw/master/docker-compose.yml

# download, install and run the cardano-wallet backend node using docker (autorestart enabled)
NETWORK=mainnet docker-compose up -d

# init sync blockchain, CPU intensive, +7GB mem, +30GB, wait +12h sync_progress
curl http://localhost:8090/v2/network/information
curl http://20.86.118.190:8090/v2/network/information # (optional: open port 8090)

# if you want to stop all cardano backend services (preserve data)
NETWORK=mainnet docker-compose down
```

### install

In [ ]:
!pip install cardano qrcode pillow psutil

### import

In [2]:
# http method
import requests
import json

# cardano method
from cardano.wallet import Wallet
from cardano.wallet import WalletService
from cardano.backends.walletrest import WalletREST

# operating system packages
import platform
import psutil
import os

# QR code
import qrcode

# other
import time

### environment

In [3]:
print(f"Cores : {psutil.cpu_count(logical=True)} ({psutil.cpu_freq().current/1000:.0f}GHz)")
print(f"Memory: {psutil.virtual_memory().total/(1024**3):.2f} GB ({psutil.virtual_memory().percent}%)")
print(f"Swap  : {os.path.getsize('/swapfile')/(1024**3):.0f} GB")
disk_size = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint).total
disk_used = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint).percent
disk_fs   = psutil.disk_partitions()[0].fstype 
print(f"Disk  : {disk_size/(1024**3):.0f} GB ({disk_used}% {disk_fs})")
print(f"System: {platform.uname().version.split('~')[1].split()[0]}")

Cores : 2 (3GHz)
Memory: 15.64 GB (70.8%)
Swap  : 8 GB
Disk  : 145 GB (82.3% ext4)
System: 18.04.1-Ubuntu


In [4]:
# installed python modules
conda_version = ! conda -V
print(f"conda       : {conda_version[0].split()[1]}")
pip_version = ! pip -V
print(f"pip         : {pip_version[0].split()[1]}")
python_version = ! python -V
print(f"python      : {python_version[0].split()[1]}")
cardano_version = ! pip list |grep -i cardano
print(f"cardano     : {cardano_version[0].split()[1]}")
requests_version = ! pip list |grep -i requests
print(f"requests    : {requests_version[0].split()[1]}")
json_version = ! pip list |grep -i json
print(f"json        : {json_version[0].split()[1]}")
qrcode_version = ! pip list |grep -ie "^qrcode "
print(f"qrcode      : {qrcode_version[0].split()[1]}")
pillow_version = ! pip list |grep -ie "^pillow "
print(f"pillow      : {pillow_version[0].split()[1]}")

conda       : 4.10.1
pip         : 21.1.2
python      : 3.8.10
cardano     : 0.7.3
requests    : 2.25.1
json        : 3.2.0
qrcode      : 7.3.1
pillow      : 8.2.0


## REST protocol example

**documentation:**  
  * [openAPI docs](https://input-output-hk.github.io/cardano-wallet/api/edge/)  
  * [guide NAcrypto youtube](https://www.youtube.com/watch?v=wyZyopHP5hA)

In [5]:
# synchronization progress of the node
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/getNetworkInformation
requests.get('http://localhost:8090/v2/network/information').text

'{"network_tip":{"time":"2021-10-21T14:58:55Z","epoch_number":297,"absolute_slot_number":43262044,"slot_number":321244},"node_era":"alonzo","node_tip":{"height":{"quantity":6399738,"unit":"block"},"time":"2021-10-21T14:58:48Z","epoch_number":297,"absolute_slot_number":43262037,"slot_number":321237},"sync_progress":{"status":"ready"},"next_epoch":{"epoch_start_time":"2021-10-22T21:44:51Z","epoch_number":298}}'

In [6]:
# generated wallet with Daedalus or Yoroi (24-words private seed phrase)
mnemonic_sentence = ["pride","assault","husband","ball","convince","canvas",
                     "mind","crane","nurse","feel","tongue","pride",
                     "scrap","inflict","split","antenna","slight","foster",
                     "piano","carpet","ranch","unveil","believe","black"]

# choose a name for the wallet
wallet_name = "a wallet name"

# spending password for the wallet
passphrase = "difficultpassword"

In [7]:
# create and restore a wallet from a mnemonic sentence
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/postWallet
requests.post(url='http://localhost:8090/v2/wallets',
              json={"name":wallet_name,
                    "mnemonic_sentence":mnemonic_sentence,
                    "passphrase":passphrase,
                    "address_pool_gap":20}).text

'{"passphrase":{"last_updated_at":"2021-10-21T14:59:04.006545724Z"},"address_pool_gap":20,"state":{"status":"syncing","progress":{"quantity":0,"unit":"percent"}},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":0,"unit":"lovelace"},"available":{"quantity":0,"unit":"lovelace"}},"name":"a wallet name","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":0,"unit":"block"},"time":"2017-09-23T21:44:51Z","epoch_number":0,"absolute_slot_number":0,"slot_number":0},"assets":{"total":[],"available":[]}}'

In [8]:
# list all wallets
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/listWallets
requests.get('http://localhost:8090/v2/wallets').text

'[{"passphrase":{"last_updated_at":"2021-10-21T14:59:04.006545724Z"},"address_pool_gap":20,"state":{"status":"syncing","progress":{"quantity":2.26,"unit":"percent"}},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":0,"unit":"lovelace"},"available":{"quantity":0,"unit":"lovelace"}},"name":"a wallet name","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":144994,"unit":"block"},"time":"2017-10-27T11:26:31Z","epoch_number":6,"absolute_slot_number":145025,"slot_number":15425},"assets":{"total":[],"available":[]}}]'

In [9]:
# wallet id
wid = "34d88843c44eb8b13a0f03a3ef387fa3b5e8733c"

In [10]:
# synchronization progress of the wallet and available quantity (1 ADA = 1.000.000 lovelace)
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/getWallet
requests.get('http://localhost:8090/v2/wallets/' + wid).text

'{"passphrase":{"last_updated_at":"2021-10-21T14:59:04.006545724Z"},"address_pool_gap":20,"state":{"status":"ready"},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":0,"unit":"lovelace"},"available":{"quantity":0,"unit":"lovelace"}},"name":"a wallet name","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":6399824,"unit":"block"},"time":"2021-10-21T15:31:55Z","epoch_number":297,"absolute_slot_number":43264024,"slot_number":323224},"assets":{"total":[],"available":[]}}'

In [11]:
# show a list of known public addresses
# https://input-output-hk.github.io/cardano-wallet/api/edge/#tag/Addresses
requests.get('http://localhost:8090/v2/wallets/' + wid + '/addresses').text

'[{"state":"used","id":"addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv","derivation_path":["1852H","1815H","0H","0","0"]},{"state":"used","id":"addr1q9m6ty3st0kzhlgg2wp8vm8fmrwxpypl2kx6snq3gtgwp39ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs6p09ug","derivation_path":["1852H","1815H","0H","1","0"]},{"state":"unused","id":"addr1q9dsnjj2ryz55ykq9kwp48hvx4cfqtfszna3lqnuzj29feaucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs20wk26","derivation_path":["1852H","1815H","0H","0","1"]},{"state":"used","id":"addr1q8cjph2s9wau69w355g29cpeaukysmn29lh9xzx93p6s6jaucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsl9lmd2","derivation_path":["1852H","1815H","0H","1","1"]},{"state":"unused","id":"addr1q8sazmfqsad2hm4j3052dj4rhlh8jn7ft2yy8z8m85al8lducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsy40dr5","derivation_path":["1852H","1815H","0H","0","2"]},{"state":"unused","id":"addr1q95k6rp7jyp0zhdm240v2rsnmgewnxhy57h8ky3aaduw0s4ucnhmu76m8kpkr24jwzc95emzqq89dzgn

In [12]:
# public wallet address
address = "addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv"

In [13]:
# generate an instance of QR code
qr = qrcode.QRCode(version=1, box_size=2, border=1)
  # version  = dimension of qr code [1-40]
  # box_size = pixel size of a black box [1-10]
  # border   = border thickness of x black boxes

# add public wallet address to this QR code
qr.add_data(address)
qr.make(fit=True)

# convert QR code into image and store
img = qr.make_image(fill='black', back_color='white')
img.save('../../image/howto_cardano/qrcode.png')

**transfer 5.0 ADA to this public wallet address (mainnet)**  
![enable outliers flag](../../image/howto_cardano/qrcode.png)  

In [14]:
# wallet synchronization progress available quantity
requests.get('http://localhost:8090/v2/wallets/' + wid).text

'{"passphrase":{"last_updated_at":"2021-10-21T14:59:04.006545724Z"},"address_pool_gap":20,"state":{"status":"ready"},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":4200000,"unit":"lovelace"},"available":{"quantity":4200000,"unit":"lovelace"}},"name":"a wallet name","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":6399847,"unit":"block"},"time":"2021-10-21T15:39:08Z","epoch_number":297,"absolute_slot_number":43264457,"slot_number":323657},"assets":{"total":[],"available":[]}}'

In [15]:
# estimate transaction fee (blockchain must be in sync with available funds)
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/postTransactionFee
transaction_data = {"passphrase":passphrase, 
                    "payments":[{"address":"Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7",
                                 "amount":{"quantity":2000000, "unit":"lovelace"}}]}
requests.post('http://localhost:8090/v2/wallets/' + wid + '/payment-fees', json=transaction_data).text

'{"estimated_min":{"quantity":168405,"unit":"lovelace"},"deposit":{"quantity":0,"unit":"lovelace"},"minimum_coins":[{"quantity":999978,"unit":"lovelace"}],"estimated_max":{"quantity":168405,"unit":"lovelace"}}'

In [16]:
# calculate transfer amount (2.0 ADA = 2000000 lovelace)
funds = 2000000             # user-desired amount to transfer (available in wallet)
fee   = 168405              # transaction cost (0.168405 ADA)
# quantity = funds - fee    # option 1: allow to empty the wallet to 0.0
quantity =  funds + fee     # option 2: we pay the fee to allow the receiver the user-desired amount
quantity

2168405

In [17]:
# send transaction from the wallet out
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/postTransaction
transaction_data = {"passphrase":passphrase,
                    "payments":[{"address":"Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7",
                                 "amount":{"quantity":quantity, "unit":"lovelace"}}]}
requests.post('http://localhost:8090/v2/wallets/' + wid + '/transactions', json=transaction_data).text

'{"status":"pending","withdrawals":[],"amount":{"quantity":2336810,"unit":"lovelace"},"inputs":[{"amount":{"quantity":4200000,"unit":"lovelace"},"address":"addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv","id":"8716217b794682c0bf74fb3698aee6a8691e61601de9837cf32807cac0260152","assets":[],"index":3}],"direction":"outgoing","fee":{"quantity":168405,"unit":"lovelace"},"outputs":[{"amount":{"quantity":2168405,"unit":"lovelace"},"address":"Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7","assets":[]},{"amount":{"quantity":1863190,"unit":"lovelace"},"address":"addr1q9a4q6m4luv3gd9f4lcv5mjhkfc7d003tw2nuq96rlu7nmducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs4dxycy","assets":[]}],"script_validity":"valid","expires_at":{"time":"2021-10-21T17:39:59Z","epoch_number":297,"absolute_slot_number":43271708,"slot_number":330908},"pending_since":{"height":{"quantity":6399850,"unit":"block"},"time":"2021-10-21T15:39:55Z","epoch_number":29

In [18]:
# wallet synchronization progress available quantity
requests.get('http://localhost:8090/v2/wallets/' + wid).text

'{"passphrase":{"last_updated_at":"2021-10-21T14:59:04.006545724Z"},"address_pool_gap":20,"state":{"status":"ready"},"balance":{"reward":{"quantity":0,"unit":"lovelace"},"total":{"quantity":1863190,"unit":"lovelace"},"available":{"quantity":1863190,"unit":"lovelace"}},"name":"a wallet name","delegation":{"next":[],"active":{"status":"not_delegating"}},"id":"34d88843c44eb8b13a0f03a3ef387fa3b5e8733c","tip":{"height":{"quantity":6399853,"unit":"block"},"time":"2021-10-21T15:40:31Z","epoch_number":297,"absolute_slot_number":43264540,"slot_number":323740},"assets":{"total":[],"available":[]}}'

In [19]:
# delete wallet from the backend (funds will remain available)
# https://input-output-hk.github.io/cardano-wallet/api/edge/#operation/deleteWallet
requests.delete('http://localhost:8090/v2/wallets/' + wid).text

''

## Python module example

**documentation:**  
  * [git repo of the cardano python module](https://github.com/emesik/cardano-python)  
  * [documentation for the cardano python module](https://cardano-python.readthedocs.io/en/latest/index.html)  

In [ ]:
# synchronization progress of the node


In [20]:
# generated wallet with Daedalus or Yoroi (24-words private seed phrase)
mnemonic_sentence = ["pride","assault","husband","ball","convince","canvas",
                     "mind","crane","nurse","feel","tongue","pride",
                     "scrap","inflict","split","antenna","slight","foster",
                     "piano","carpet","ranch","unveil","believe","black"]
mnemonic = " ".join(mnemonic_sentence)

# choose a name for the wallet
wallet_name = "a wallet name"

# spending password for the wallet
passphrase = "difficultpassword"

In [21]:
# create and restore a wallet from a mnemonic sentence
ws = WalletService(WalletREST(port=8090))                  # wallet service REST backend
wid = ws.create_wallet(wallet_name, mnemonic, passphrase)  # recover wallet on backend

In [22]:
# list all wallets
ws.wallets()

In [23]:
# wallet id
wid

'34d88843c44eb8b13a0f03a3ef387fa3b5e8733c'

In [24]:
# open the wallet (with id)
wal = ws.wallet(wid, passphrase)

In [25]:
%%time
# synchronization progress of the wallet
while wal.sync_progress() < 1.0:
            time.sleep(12)

CPU times: user 5.01 s, sys: 426 ms, total: 5.44 s
Wall time: 30min 2s


In [31]:
# check available wallet quantity (ADA)
wal.balance()  # reward — the amount received as staking interest

Balance(total=Decimal('1.863190'), available=Decimal('1.863190'), reward=Decimal('0.000000'))

In [32]:
# show a list of known public addresses
wal.addresses()

[addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv,
 addr1q9m6ty3st0kzhlgg2wp8vm8fmrwxpypl2kx6snq3gtgwp39ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs6p09ug,
 addr1q9dsnjj2ryz55ykq9kwp48hvx4cfqtfszna3lqnuzj29feaucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs20wk26,
 addr1q8cjph2s9wau69w355g29cpeaukysmn29lh9xzx93p6s6jaucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsl9lmd2,
 addr1q8sazmfqsad2hm4j3052dj4rhlh8jn7ft2yy8z8m85al8lducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsy40dr5,
 addr1q9a4q6m4luv3gd9f4lcv5mjhkfc7d003tw2nuq96rlu7nmducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs4dxycy,
 addr1q95k6rp7jyp0zhdm240v2rsnmgewnxhy57h8ky3aaduw0s4ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhss2rly5,
 addr1qxfht2ar6fydjf0n27ccc796zmqmqk244mmpvp06ww95uf4ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs8zp0as,
 addr1qym3t8e9462k7mvhargrxs0hn2dn3kpwdrt5kz3tr6d7cc4ucnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhs0e4tj8,
 addr1qxsdtlt6ajdf4p3h6nzpddx3zqxw7l4dvt375tdc

In [33]:
# public wallet address
address = "addr1qx0303ays44x6u6c3v5uyn448xxmts0qahwhs5tgd9h6fvducnhmu76m8kpkr24jwzc95emzqq89dzgnupg6v0947uhsek6skv"

In [34]:
# generate an instance of QR code
qr = qrcode.QRCode(version=1, box_size=2, border=1)
  # version  = dimension of qr code [1-40]
  # box_size = pixel size of a black box [1-10]
  # border   = border thickness of x black boxes

# add public wallet address to this QR code
qr.add_data(address)
qr.make(fit=True)

# convert QR code into image and store
img = qr.make_image(fill='black', back_color='white')
img.save('../../image/howto_cardano/qrcode.png')

**optional:** transfer more ADA to this public wallet address (mainnet)  
![enable outliers flag](../../image/howto_cardano/qrcode.png) 

In [ ]:
# check available wallet quantity (ADA)
wal.balance()

In [35]:
# estimate transaction fee (blockchain must be in sync with available funds)
(min_fee, max_fee) = wal.estimate_fee([("Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7", wal.balance()[0])])
min_fee

Decimal('0.168405')

In [36]:
# calculate transfer amount
funds = wal.balance()[0]    # user-desired amount to transfer (available in wallet)
fee   = min_fee             # transaction cost (0.168405 ADA)
quantity = funds - fee      # option 1: allow to empty the wallet to 0.0
# quantity =  funds + fee   # option 2: we pay the fee to allow the receiver the user-desired amount
quantity

Decimal('1.694785')

In [37]:
# send transaction from the wallet
tx = wal.transfer(address="Ae2tdPwUPEZMgbgGHr4tM6pV6DSwdjNzVon16Qr6YSGeQW6xnmqbwXfh3o7",
                  amount=quantity,
                  passphrase=passphrase)

In [38]:
# check available wallet quantity (0.0 ADA)
wal.balance()

Balance(total=Decimal('0.000000'), available=Decimal('0.000000'), reward=Decimal('0.000000'))

In [39]:
tx.amount_out

Decimal('1.694785')

In [40]:
tx.fee

Decimal('0.168405')

In [ ]:
# delete wallet from the backend (funds will remain available)
wal.delete()